### Weiqing Kong

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
from collections import Counter

In [2]:
sheet_id = '16i38oonuX1y1g7C_UAmiK9GkY7cS-64DfiDMNiR41LM'
sheet_name = 'Shopify'
URL = 'https://docs.google.com/spreadsheets/d/{0}/gviz/tq?tqx=out:csv&sheet={1}'.format(sheet_id,sheet_name)

In [3]:
df = pd.read_csv(URL);df

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11
...,...,...,...,...,...,...,...
4995,4996,73,993,330,2,debit,2017-03-30 13:47:17
4996,4997,48,789,234,2,cash,2017-03-16 20:36:16
4997,4998,56,867,351,3,cash,2017-03-19 5:42:42
4998,4999,60,825,354,2,credit_card,2017-03-16 14:51:18


In [4]:
#How the wrong AOV calculated
df['order_amount'].mean()

3145.128

In [5]:
#Define a function to detect outliers
def outlier_dect(x,y):
    q1 = np.quantile(x,0.25)
    q3 = np.quantile(x,0.75)
    mean = np.mean(x)
    std = np.std(x)
    IQR = q3 - q1
    #z-test hypothesis
    if y == 'z_test':
        outlier = [j for i,j in enumerate(x) if scipy.stats.norm.sf(abs((j-mean)/std)) < 0.05 ]
        outlier_index = [i for i,j in enumerate(x) if scipy.stats.norm.sf(abs((j-mean)/std)) < 0.05]
    #quantile method
    if y == 'quantile':
        outlier = [j for i,j in enumerate(x) if (j > q3 + 1.5*IQR) or (j < q1 - 1.5*IQR)]
        outlier_index = [i for i,j in enumerate(x) if (j > q3 + 1.5*IQR) or (j < q1 - 1.5*IQR)]
    return outlier, outlier_index

In [6]:
#add a new column that shows price of a pair of sneaker in each store
df['single_price'] = df['order_amount']/df['total_items'];df

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,single_price
0,1,53,746,224,2,cash,2017-03-13 12:36:56,112.0
1,2,92,925,90,1,cash,2017-03-03 17:38:52,90.0
2,3,44,861,144,1,cash,2017-03-14 4:23:56,144.0
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37,156.0
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11,156.0
...,...,...,...,...,...,...,...,...
4995,4996,73,993,330,2,debit,2017-03-30 13:47:17,165.0
4996,4997,48,789,234,2,cash,2017-03-16 20:36:16,117.0
4997,4998,56,867,351,3,cash,2017-03-19 5:42:42,117.0
4998,4999,60,825,354,2,credit_card,2017-03-16 14:51:18,177.0


In [7]:
#calculate each shop's single price of selling sneakers and verify the correctness of data
shop_id = np.unique(df['shop_id'])
shop_price = [[i] for i in shop_id]
for i,j in enumerate(df['shop_id']):
    shop_price[j-1].append(df['single_price'][i])
shop_price = [np.unique(i) for i in shop_price]
for i in shop_price:
    print('StoreID: ', int(i[0]), '|Sneaker price: ',i[1])

StoreID:  1 |Sneaker price:  158.0
StoreID:  2 |Sneaker price:  94.0
StoreID:  3 |Sneaker price:  148.0
StoreID:  4 |Sneaker price:  128.0
StoreID:  5 |Sneaker price:  142.0
StoreID:  6 |Sneaker price:  187.0
StoreID:  7 |Sneaker price:  112.0
StoreID:  8 |Sneaker price:  132.0
StoreID:  9 |Sneaker price:  118.0
StoreID:  10 |Sneaker price:  148.0
StoreID:  11 |Sneaker price:  184.0
StoreID:  12 |Sneaker price:  201.0
StoreID:  13 |Sneaker price:  160.0
StoreID:  14 |Sneaker price:  116.0
StoreID:  15 |Sneaker price:  153.0
StoreID:  16 |Sneaker price:  156.0
StoreID:  17 |Sneaker price:  176.0
StoreID:  18 |Sneaker price:  156.0
StoreID:  19 |Sneaker price:  163.0
StoreID:  20 |Sneaker price:  127.0
StoreID:  21 |Sneaker price:  142.0
StoreID:  22 |Sneaker price:  146.0
StoreID:  23 |Sneaker price:  156.0
StoreID:  24 |Sneaker price:  140.0
StoreID:  25 |Sneaker price:  130.0
StoreID:  26 |Sneaker price:  176.0
StoreID:  27 |Sneaker price:  169.0
StoreID:  28 |Sneaker price:  164.0
St

In [8]:
price = [i[1] for i in shop_price]
outlier_dect(price, 'quantile')

([352.0, 25725.0], [41, 77])

The outliers here are shop_id 42 with a single price 352 and shop_id 78 with a single price 25725.

In [9]:
#outlier index determined by price of the model sold in each store
price_outlier_index = list(np.where(df['shop_id'] == 42)[0]) + list(np.where(df['shop_id'] == 78)[0])

In [10]:
#outlier index determined by order amount 
amount_outlier, amount_outlier_index = outlier_dect(df['order_amount'],'quantile')

In [11]:
#Union of price outliers and order amount outliers
outlier_index = np.unique(price_outlier_index + amount_outlier_index);outlier_index

array([  15,   40,   60,   99,  136,  160,  219,  222,  259,  264,  308,
        409,  490,  493,  511,  520,  522,  617,  651,  691,  737,  742,
        771,  834,  835,  879,  938,  979,  994, 1056, 1104, 1123, 1150,
       1193, 1204, 1256, 1259, 1362, 1364, 1367, 1384, 1419, 1436, 1452,
       1471, 1484, 1503, 1512, 1520, 1529, 1562, 1563, 1602, 1628, 1764,
       1911, 1923, 1929, 1946, 1948, 1962, 2003, 2018, 2032, 2039, 2043,
       2053, 2127, 2136, 2140, 2153, 2270, 2273, 2297, 2307, 2353, 2386,
       2389, 2452, 2491, 2492, 2494, 2495, 2512, 2548, 2560, 2564, 2609,
       2670, 2690, 2757, 2766, 2773, 2818, 2821, 2835, 2906, 2922, 2967,
       2969, 2987, 3073, 3077, 3085, 3101, 3117, 3151, 3167, 3202, 3252,
       3332, 3403, 3428, 3438, 3440, 3513, 3517, 3532, 3538, 3609, 3651,
       3697, 3705, 3724, 3780, 3865, 3903, 3927, 3966, 3998, 4040, 4056,
       4079, 4141, 4192, 4231, 4294, 4295, 4311, 4326, 4412, 4420, 4421,
       4490, 4505, 4512, 4523, 4554, 4574, 4580, 45

In [12]:
#Drop the outliers and reindex
df1 = df.copy()
df_outlier_dropped = df1.drop(index = outlier_index)
df_outlier_dropped.index = list(np.arange(df_outlier_dropped.shape[0]));df_outlier_dropped

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,single_price
0,1,53,746,224,2,cash,2017-03-13 12:36:56,112.0
1,2,92,925,90,1,cash,2017-03-03 17:38:52,90.0
2,3,44,861,144,1,cash,2017-03-14 4:23:56,144.0
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37,156.0
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11,156.0
...,...,...,...,...,...,...,...,...
4826,4996,73,993,330,2,debit,2017-03-30 13:47:17,165.0
4827,4997,48,789,234,2,cash,2017-03-16 20:36:16,117.0
4828,4998,56,867,351,3,cash,2017-03-19 5:42:42,117.0
4829,4999,60,825,354,2,credit_card,2017-03-16 14:51:18,177.0


In [13]:
aov = np.mean(df_outlier_dropped['order_amount']);aov

292.4303456841234

The number 3145.13 was calulated directly by averaging the order amount of each line. The result looks inreasonable because, there are some outliers exist. By the outlier detection function with z-test and quantile methods, the function can output those outliers that are much larger than other order amounts due to either large 'total_items' values or high single price.

The metric used above is to conclude outliers in two different ways:<br>
1) Caluclate single item price for each shop and exclude those shops that have unusual single item prices. <br>
The result are the shop_id 42 with a single price 352 and shop_id 78 with a single price 25725.<br>
2) Use outlier detection function to find outliers in the column 'order_amount' that could include some other unusual orders from shops apart from the shop 42 and the shop 78. <br>
The result used the quantile method instead of z-test hypothesis because, the significance level is diffcult to determine - even the result computed using 10% significance include large amount orders.  

By dropping the outliers discussed above, the final AOV value is 292.43.